In [ ]:
import cx_Oracle
import pandas as pd
import numpy as np
import sys
import os
import importlib
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datastory import DataStory

from google.cloud import secretmanager

sys.path.append('../lib')
import pandas_utils
import pesys_utils
import autograd_convenience
import utils

importlib.reload(pandas_utils)
importlib.reload(pesys_utils)
importlib.reload(autograd_convenience)
importlib.reload(utils)

utils.set_secrets_as_env()

In [ ]:
try:
    con = pesys_utils.open_pen_connection()
except cx_Oracle.DatabaseError:
    con.close()
    con = pesys_utils.open_pen_connection()
    
df_list = pandas_utils.pandas_from_sql('../sql/forslag_auto.sql', con, tuning = 10000)
#df_list = pandas_utils.pandas_from_sql('../sql/v8_kun_norge.sql', con)
con.close()
#df_list["ÅR-TERTIAL"] = df_list["ÅR-TERTIAL"].apply(lambda x: x[:4] + "-" + x[4:])

In [ ]:
df_list.head()

In [ ]:
from google.cloud.bigquery import Client, LoadJobConfig

table_id = f'pensjon-saksbehandli-prod-1f83.vedtak.vedtak_automatisering_v2'
job_config = LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",
)

client = Client(project="pensjon-saksbehandli-prod-1f83")

job = client.load_table_from_dataframe(df_list, table_id, job_config=job_config)
job.result()

In [ ]:
tmp = df_list[df_list.BEHANDLER=="BPEN068"]
tmp[tmp.BATCH != "Batch"]

In [ ]:
df_list.KRAVTYPE.value_counts()